# What are we doing?

## Objectives

+ Construct a cross-validation pipeline.
+ Use cross-validation to evaluate different hyperparameter performance.
+ Perform grid search for systemic evaluation.
+ Store and manage results.

## Procedure

The diagram below, taken from Scikit Learn's documentation, shows the procedure that we will follow:

![](./images/05_grid_search_workflow.png)


+ System requriements:
    
    - Automation: the system should operate automatically with the least amount of supervision. 
    - Replicability: changes to code and (arguably) data should be logged and controled. Randomness should also be controlled (random seeds, etc.)
    - Persistence: persist results for later analysis.


## What is a Hyperparameter?

+ Generally speaking, hyperparameters are parameters that control the learning process: regularization weights, learning rate, entropy/gini metrics, etc. 
+ Hyperparameters will drive the behaviour and performance of a model. Model selection is intimately related with hyperparameter tuning. 
+ Selection critieria are based on performance evaluation and, to get better performance estimates, we use cross-validation.

## Searching the Hyperparameter Grid

+ To address the automation requirement, we could use `GridSearchCV()`, which is a self-contained function for performing a Grid Search over a hyperparameter space.
+ To "Search the Hyperparameter Grid" exhaustively means that we will consider all possible combination of hyperparameter values in the search space and evaluate the model using those hyperparams. For example, if we have two parameters that we are exploring, kernel (takes values "rbf" and "poly") and C (takes values 1.0 and 0.5), then this grid would be the combinations:

    + (rbf, 1.0)
    + (rbf, 0.5)
    + (poly, 1.0)
    + (poly, 0.5)

+ Under each combination, we perform CV and evaluate the model's performance.

# Setup

We start with [Give me some credit](https://www.kaggle.com/c/GiveMeSomeCredit) data that we used in the previous session.

In [ ]:
# Load environment variables
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv(dotenv_path='.env')

# System & OS utilities
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add project source directory
sys.path.append(os.getenv('SRC_DIR'))

# Scikit-learn: Preprocessing, Pipelines, and Model Selection
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import (
    train_test_split, 
    GridSearchCV, 
    ParameterGrid
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import (
    classification_report, 
    roc_auc_score, 
    roc_curve, 
    confusion_matrix
)

# Parallel processing & Progress Bar
from tqdm import tqdm
from joblib import Parallel, delayed

# Debugging & Logging
import logging

# Load environment variables
log_dir = os.getenv('LOG_DIR')
db_url = os.getenv('DB_URL')
tickers_file = os.getenv('TICKERS')

print(f"Log Directory: {log_dir}")
print(f"Tickers File: {tickers_file}")
print(f"Database URL: {db_url}")
print("Current working directory:", os.getcwd())
print("DEBUG: DB_URL =", os.getenv("DB_URL"))


In [4]:
df = df_raw.drop(columns = ["Unnamed: 0"]).rename(
    columns = {
        'SeriousDlqin2yrs': 'delinquency',
        'RevolvingUtilizationOfUnsecuredLines': 'revolving_unsecured_line_utilization', 
        'age': 'age',
        'NumberOfTime30-59DaysPastDueNotWorse': 'num_30_59_days_late', 
        'DebtRatio': 'debt_ratio', 
        'MonthlyIncome': 'monthly_income',
        'NumberOfOpenCreditLinesAndLoans': 'num_open_credit_loans', 
        'NumberOfTimes90DaysLate':  'num_90_days_late',
        'NumberRealEstateLoansOrLines': 'num_real_estate_loans', 
        'NumberOfTime60-89DaysPastDueNotWorse': 'num_60_89_days_late',
        'NumberOfDependents': 'num_dependents'
    }
).assign(
    high_debt_ratio = lambda x: (x['debt_ratio'] > 1)*1,
    missing_monthly_income = lambda x: x['monthly_income'].isna()*1,
    missing_num_dependents = lambda x: x['num_dependents'].isna()*1, 
)


Use a simple pipeline composed of:

+ Preprocessing steps.
+ Logistic Regression classifier.

We will explore the hyperparameter sapce by evaluating different regularization strategies and parameters.

In [6]:
class DataFrameWrapper(BaseEstimator, TransformerMixin):
    """Ensures transformed NumPy arrays are converted back to Pandas DataFrames while keeping expected column names"""

    def __init__(self):
        self.feature_names = None  

    def fit(self, X, y=None):
        """Store feature names if input is a DataFrame"""
        if hasattr(X, "columns"):
            self.feature_names = list(X.columns)
        return self

    def transform(self, X):
        """Convert array back to DataFrame with stored feature names"""
        if self.feature_names is not None:
            return pd.DataFrame(X, columns=self.feature_names)
        else:
            return pd.DataFrame(X)


In [8]:
# Define feature matrix X and target variable Y
X = df.drop(columns='delinquency')
Y = df['delinquency']

# Ensure X_train and X_test are DataFrames before training
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)


In [ ]:
num_cols = ['revolving_unsecured_line_utilization', 'age',
       'num_30_59_days_late', 'debt_ratio', 'monthly_income',
       'num_open_credit_loans', 'num_90_days_late', 'num_real_estate_loans',
       'num_60_89_days_late', 'num_dependents', 
       # Although expressed as numbers, these columns are boolean:
       # 'high_debt_ratio',
       # 'missing_monthly_income', 
       # 'missing_num_dependents' 
       ]

pipe_num_simple = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('standardizer', StandardScaler())
])

ctransform_simple= ColumnTransformer(
    [('numeric_simple', pipe_num_simple, num_cols),],
    remainder='passthrough',force_int_remainder_cols=False)

pipe_lr = Pipeline([
    ("scaler", StandardScaler()),  # Standardizes numerical features
    ("preprocess", ctransform_simple),  # Applies transformations
    ("clf", LogisticRegression())  # Model
])


pipe_lr


Obtain the parameters of the pipeline with `.get_params()`.

In [ ]:
pipe_lr.get_params()


## Setup the Splitting Strategy

In [11]:
# X = df.drop(columns = 'delinquency')
# Y = df['delinquency']

# scoring = ['neg_log_loss', 'roc_auc', 'f1', 'accuracy', 'precision', 'recall']

# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)



To perform the Grid Search we need to define a parameter grid:

- A parameter grid defines all of the combinations of parameters that we need to explore.
- The function `GridSearchCV()` performs an exhaustive search of parameter combinations.
- The parameter grid is defined as a dictionary of lists:

    * Each entry's key is the name of the parameter.
    * Each entry's value is the list of values that we would like to explore.

In [12]:
solver_penalty_map = {
    "lbfgs": {"penalty": ["l2"]},  # lbfgs only supports L2
    "liblinear": {"penalty": ["l1", "l2"]},  # Supports L1 and L2
    "saga": {"penalty": ["l1", "l2", "elasticnet"], "l1_ratio": [0.1, 0.5, 0.9]},  # Needs l1_ratio for elasticnet
    "newton-cg": {"penalty": ["l2"]},  # Only supports L2
    "sag": {"penalty": ["l2"]},  # Only supports L2
}

param_grid = []
for solver, params in solver_penalty_map.items():
    grid = {
        "clf__solver": [solver], 
        "clf__penalty": params["penalty"],
        "clf__C": [0.001, 0.01, 0.1, 0.5, 1.0]
    }
    if "l1_ratio" in params:
        grid["clf__l1_ratio"] = params["l1_ratio"]
    param_grid.append(grid)


In [13]:
# param_grid = {
#     "clf__C": [0.001, 0.01, 0.1, 0.5, 1.0],  # Regularization strength
#     "clf__penalty": ["l1", "l2"]#,  # Penalty type
#     #"clf__solver": ["liblinear", "saga"],  # Only solvers that support both l1 and l2
# }


Some key inputs to [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) are:

+ `estimator`: the pipeline or classifier that we are tuning.
+ `param_grid`: the parameter grid defined as a dictionary of lists described above.
+ `n_jobs`: settings for parallel computation.
+ `refit`: options for refitting the model using the best-performing configuration.

In [14]:
scoring = ['neg_log_loss', 'roc_auc', 'f1', 'accuracy', 'precision', 'recall']


In [ ]:
grid_cv = GridSearchCV(
    estimator=pipe_lr, 
    param_grid=param_grid, 
    scoring=scoring, 
    cv=5,
    refit="neg_log_loss",
    n_jobs=-1,
    verbose=3  # Enables built-in progress updates
)

grid_cv.fit(X_train, Y_train)


Access the cross-validation results using the property `.cv_results_`:

In [ ]:
res = grid_cv.cv_results_
res = pd.DataFrame(res)
res.columns

res[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_clf__C', 'param_clf__penalty', 'param_clf__solver', 'params',
       'mean_test_neg_log_loss',
       'std_test_neg_log_loss', 'rank_test_neg_log_loss']].sort_values('rank_test_neg_log_loss')


In [ ]:
res = pd.DataFrame(grid_cv.cv_results_)
print(res.columns)  # Verify available columns


In [ ]:
res


In [ ]:
res = pd.DataFrame(grid_cv.cv_results_)

# Drop invalid results (failed fits)
res_filtered = res.dropna(subset=["mean_test_neg_log_loss"])

# Display best results
print("\n🔹 Best Results:")
print(res_filtered.nsmallest(5, "rank_test_neg_log_loss"))


Access the best-performing configuration:

In [ ]:
grid_cv.best_params_


In [ ]:
grid_cv.best_estimator_


The best-performing classifier (pipeline) trained on the complete training set is:

# Tracking GridSearchCV Experiments

+ We can expand our infrastructure for hyperparameter tuning across various models.
+ The plan:

    - Create a model ingredient to obtain the classifier object.
    - Create experiment param grids in json files to organize our parameter grids.
    - Schedule the experiments.


## The Design

<div>
<img src=./images/05_experiment_setup.png width="75%">
</div>

Explore the code in `./05_src/credit_experiment.py` and `./05_src/credit_model_ingredient.py`:

+ `credit_model_ingredient.py` implements a function that returns a model given a string. This way, we can parametrize models in the experiment.
+ `credit_experiment.py` is modularized version of our previous file, `credit_experiment_nb.py` which only worked with Naive Bayes classifier.
+ The experiment is now further *modularized*: there are ingredients for most components and it can be broken down even more depending on the evolution of the model.

## Running Experiments from the Command Line

Access the experiment through the [Command Line Interface](https://sacred.readthedocs.io/en/stable/command_line.html).

```
cd src  # if required
python credit_experiment.py
```

We can also change the parameters of the experiment. For instance, using the same code, we can run an experiment with a logistic regression classifier using a basic (not power) preprocessing pipeline:

```
python .\credit_experiment.py with 'preprocessing="basic"' 'model="LogisticRegression"'
```

# A Few Notes About Sacred

+ Sacred is a powerful tool, but it is only the beginning.  
+ Sacred is useful in keeping track of experiments within a limited scope: it is not a project management tool.
+ It works well in SQL environments, but handling hyperparameters can be painful.
+ The natural backend is MongoDB, however not all workplaces have running instances.


## Experiment Schema

The database schema implemented by sacred is shown below. The schema is a useful representation of the code and setup of an experiment. The package offers a [metrics API](https://sacred.readthedocs.io/en/stable/collected_information.html#metrics-api), but we have decided to extend the framework with a few ad-hoc tables with performance metrics. 

The database backend is a database like any other: you can query it with Python, R, or PowerBI.

+ Server is located in localhost port 5432.
+ User and password are in the .env file in `./05_src/db/`.

<div>
<img src=./images/05_sacred_sql_schema.png width="40%">
</div>